In [8]:
from utils import MysqlConnector
conn = MysqlConnector()

In [9]:
conn.execute('''USE source;''')
conn.execute('DROP TABLE IF EXISTS sales_order')
conn.execute('''
CREATE TABLE sales_order
( order_number INT
, customer_number INT 
, product_code INT
, order_date DATE
, entry_date DATE
, order_amount DECIMAL(10,2)) 
;
''')
conn.execute('''
INSERT INTO sales_order VALUES
  (201, 1, 1, '2013-02-06', '2013-02-06', 1000)
, (202, 2, 1, '2013-02-06', '2013-02-06', 1000)
, (203, 3, 1, '2013-02-06', '2013-02-06', 4000)
, (204, 4, 1, '2013-02-06', '2013-02-06', 4000)
;
''')

In [10]:
conn.execute('USE dw;')
conn.execute('''
INSERT INTO order_dim VALUES
  (NULL, 200, CURRENT_DATE, '9999-12-31')
, (NULL, 201, CURRENT_DATE, '9999-12-31')
, (NULL, 202, CURRENT_DATE, '9999-12-31')
, (NULL, 203, CURRENT_DATE, '9999-12-31')
, (NULL, 204, CURRENT_DATE, '9999-12-31')
, (NULL, 205, CURRENT_DATE, '9999-12-31')
, (NULL, 206, CURRENT_DATE, '9999-12-31')
, (NULL, 207, CURRENT_DATE, '9999-12-31')
, (NULL, 208, CURRENT_DATE, '9999-12-31')
, (NULL, 209, CURRENT_DATE, '9999-12-31')
, (NULL, 210, CURRENT_DATE, '9999-12-31')
, (NULL, 211, CURRENT_DATE, '9999-12-31')
, (NULL, 212, CURRENT_DATE, '9999-12-31')
, (NULL, 213, CURRENT_DATE, '9999-12-31')
, (NULL, 214, CURRENT_DATE, '9999-12-31')
, (NULL, 215, CURRENT_DATE, '9999-12-31')
, (NULL, 216, CURRENT_DATE, '9999-12-31')
, (NULL, 217, CURRENT_DATE, '9999-12-31')
, (NULL, 218, CURRENT_DATE, '9999-12-31')
, (NULL, 219, CURRENT_DATE, '9999-12-31')
, (NULL, 220, CURRENT_DATE, '9999-12-31')
, (NULL, 221, CURRENT_DATE, '9999-12-31')
, (NULL, 222, CURRENT_DATE, '9999-12-31')
, (NULL, 223, CURRENT_DATE, '9999-12-31')
;             
''')
conn.execute('''
INSERT INTO date_dim VALUES
  (NULL, '2013-02-06', 'February', 2, 6, 2013, CURRENT_DATE, '9999-12-31')
, (NULL, '2013-02-10', 'February', 2, 6, 2013, CURRENT_DATE, '9999-12-31')
  ;          
''')
conn.execute('USE source;')
conn.execute('''
INSERT INTO sales_order VALUES
  (210, 1, 1, '2013-02-06', '2013-02-06', 1000)
, (211, 2, 2, '2013-02-06', '2013-02-06', 1000)
, (212, 3, 2, '2013-02-06', '2013-02-06', 4000)
, (213, 4, 4, '2013-02-06', '2013-02-06', 3500)
, (214, 5, 5, '2013-02-06', '2013-02-06', 5000)
, (215, 6, 5, '2013-02-06', '2013-02-06', 1700)
, (216, 7, 6, '2013-02-06', '2013-02-06', 4600)
, (217, 7, 7, '2013-02-06', '2013-02-06', 3600)
, (218, 8, 8, '2013-02-06', '2013-02-06', 1700)
, (219, 8, 9, '2013-02-06', '2013-02-06', 1200)
, (220, 2, 10, '2013-02-06', '2013-02-06', 4400)
, (221, 10, 10, '2013-02-06', '2013-02-06', 14000)
, (222, 2, 2, '2013-02-06', '2013-02-06', 1000)
, (223, 3, 2, '2013-02-06', '2013-02-06', 4000)
, (224, 4, 4, '2013-02-06', '2013-02-06', 3510)
, (225, 5, 5, '2013-02-06', '2013-02-06', 5230)
, (226, 6, 5, '2013-02-06', '2013-02-06', 1450)
, (227, 7, 6, '2013-02-06', '2013-02-06', 440)
, (228, 7, 7, '2013-02-06', '2013-02-06', 3430)
, (229, 8, 8, '2013-02-06', '2013-02-06', 1670)
, (230, 8, 9, '2013-02-06', '2013-02-06', 12300)
, (231, 2, 10, '2013-02-06', '2013-02-06', 14900)
, (232, 10, 10, '2013-02-06', '2013-02-06', 15600)
, (233, 10, 10, '2013-02-06', '2013-02-06', 12300)
;             
''')

In [11]:
conn.execute('USE source;')
conn.execute('''
INSERT INTO dw.sales_order_fact
SELECT
  a.order_number
, c.customer_sk
, d.product_sk
, e.date_sk
, a.order_amount
FROM
  sales_order a
, dw.order_dim b
, dw.customer_dim c
, dw.product_dim d
, dw.date_dim e
WHERE
 a.order_number = b.order_number
AND a.customer_number = c.customer_number
AND a.product_code = d.product_code
AND a.order_date = e.date
;          
''')

In [15]:
conn.execute("USE dw;")
result = conn.fetch('''
SELECT
                    d.year,
    c.customer_name,
    SUM(o.order_amount) AS total
FROM
    sales_order_fact o
INNER JOIN date_dim d ON
    o.order_date_sk = d.date_sk
INNER JOIN customer_dim c ON
    o.customer_sk = c.customer_sk
WHERE d.month_name = 'February' and d.year = (SELECT MAX(year) FROM date_dim)
GROUP BY
    d.year,
    c.customer_sk
ORDER BY
    total desc;
''')

Total de compras por cliente no último mês de fevereiro

In [16]:
result

[(2013, 'Bracows', Decimal('14000.00')),
 (2013, 'Medium Retailers', Decimal('12000.00')),
 (2013, 'Loney Clients', Decimal('8200.00')),
 (2013, 'Good Companies', Decimal('7500.00')),
 (2013, 'Small Stores', Decimal('7400.00')),
 (2013, 'Wonderful Shops', Decimal('5000.00')),
 (2013, 'Iron Mount', Decimal('2900.00')),
 (2013, 'Big Customers', Decimal('2000.00')),
 (2013, 'Loyal Clients', Decimal('1700.00'))]